In [2]:
%cd ..

a:\Phong\Crawl_buffett


In [3]:
import pdfplumber

In [4]:
import tabula, os, re
import pdfplumber
import os
import pandas as pd
import numpy as np
import warnings
from src.ocrpdf import ocr_pdf, convert_pdf_to_text
import PyPDF2
import warnings

warnings.simplefilter("ignore", UserWarning)


class GetDividend:
    def __init__(self,
            path_save:str = 'Data',
            ):
        self.path_save = path_save

    def get_dividend_table(self, 
                           tables, 
                           year="Any", 
                           table_id:int = 4):
        df = pd.DataFrame(tables[table_id])
        df.dropna(subset=df.columns[0], inplace=True)
        df.dropna(axis=1, how="all", inplace=True)
        df.columns = np.arange(len(df.columns))
        df.drop(columns=df.columns[5:], inplace=True)
        df.dropna(axis=0, how="all", inplace=True)
        df = df.reset_index(drop=True)
        df = df.iloc[[1]]
        df.columns = [year, "Q1", "Q2", "Q3", "Q4"]
        df[year].iloc[0] = year
        if (pd.isna(df.iloc[0]) == False).all() and (
            df.iloc[0, 1:].str.len() < 10
        ).all():
            return df.reset_index(drop=True)
        raise Exception("Don't have dividend table'")
        
    def get_dividend_table_from_pdf(self, 
                                    id_company : int = 1301,
                                    year: str = '2022', 
                                    quy: str = 'Q4'):
        for file in os.listdir(self.path_save + f"Data/{id_company}/PDF"):
            if file.startswith(f"{year}_{quy}") and "(訂正)" not in file:
                file_name = file
                path_of_file = self.path_save + f"Data/{id_company}/PDF/{file_name}"
                try:
                    file_name = file
                    path_of_file = self.path_save + f"Data/{id_company}/PDF/{file_name}"
                    pdf = pdfplumber.open(path_of_file)
                    page = pdf.pages[0]
                    text = page.extract_text()
                    # text = convert_pdf_to_text(path_of_file)
                    text = text.replace(" ", "")
                    df = self.get_dividend_table(tables, year)
                    if len(df.index) == 1:
                        return df
                except:
                    try:
                        ocr_file = path_of_file.replace(".pdf", "_ocr.pdf")
                        if os.path.exists(ocr_file) == False:
                            ocr_pdf(path_of_file)
                        tables = tabula.read_pdf(
                            ocr_file, pages="all", multiple_tables=True, silent=True
                        )
                        df = self.get_dividend_table(tables, year)
                        if len(df.index) == 1:
                            return df
                    except:
                        print("This year have the bug", year)
                        try:
                            df = self.get_dividend_table(tables, year, table_id=3)
                            if len(df.index) == 1:
                                return df
                        except:
                            pass
                return pd.DataFrame(
                    {year: year, "Q1": "B", "Q2": "B", "Q3": "B", "Q4": "B"}, index=[0]
                )

        return pd.DataFrame(
            {year: year, "Q1": "N/A", "Q2": "N/A", "Q3": "N/A", "Q4": "N/A"}, index=[0]
        )

    def get_dividend_table_dat(self, year, text, text_replace, idx_key):
        num_line = text_replace[: idx_key + 1].count('\n')
        for _ in range(num_line):
            idx = text.find('\n')
            text = text[idx + 1:] # text cuoi cung = key_word ...
            
        print(text)
        num_line1 = text[:text.find('月')].count('\n')
        for _ in range(num_line1 + 1):
            idx = text.find('\n')
            text = text[idx + 1:]

        text_dvd = text[text.find('期') +2: text.find('\n')].strip()
        print(text_dvd)
        lst_dvd = text_dvd.split(" ")
        print(year, lst_dvd)
        if len(lst_dvd) >= 4 and len(lst_dvd) < 10:
            return pd.DataFrame(
                {year: year, "Q1": lst_dvd[0], "Q2": lst_dvd[1], "Q3": lst_dvd[2], "Q4": lst_dvd[3]}, index=[0]
            )
        elif len(lst_dvd) >= 10:
            try:
                check = int(lst_dvd[1])
                #Truong hop: ― 1 50 ― 1 50 3 00 857 54.1 1.7
                return pd.DataFrame(
                    {year: year, "Q1": lst_dvd[0],
                                "Q2": lst_dvd[1] + '.' + lst_dvd[2],
                                "Q3": lst_dvd[3],
                                "Q4": lst_dvd[4] + '.' + lst_dvd[5]}, index=[0]
                )
            except:
                try:
                    check = int(lst_dvd[4])
                    #Truong hop:  － － － 1 00 1 00 27 △109.9 1.7
                    return pd.DataFrame(
                        {year: year, "Q1": lst_dvd[0],
                                    "Q2": lst_dvd[1],
                                    "Q3": lst_dvd[2],
                                    "Q4": lst_dvd[3] + '.' + lst_dvd[4]}, index=[0]
                    )
                except:
                    #Truong hop ― ― 1 50 ― ― 1 50 3 00 857 54.1 1.7
                    return pd.DataFrame(
                        {year: year, "Q1": lst_dvd[0] + '.' + lst_dvd[1],
                                    "Q2": lst_dvd[2] + '.' + lst_dvd[3],
                                    "Q3": lst_dvd[4] + '.' + lst_dvd[5],
                                    "Q4": lst_dvd[6] + '.' + lst_dvd[7]}, index=[0]
                    )
                
        return pd.DataFrame({year: year, "Q1": "_", "Q2": "_", "Q3": "_", "Q4": "_"}, index=[0])
        
        # loi dich: 2020453A HA ーー 0.00 — 10.00 10.00 8,804 35.2 1.4--- 9831 2019 q4
        
    def get_dividend_table_from_pdf_dat(self, 
                                    id_company : int = 1301,
                                    year: str = '2022', 
                                    quy: str = 'Q4'):
        for file in os.listdir(self.path_save + f"Data/{id_company}/PDF"):
            if file.startswith(f"{year}_{quy}") and "(訂正)" not in file:
                try:
                    file_name = file
                    path_of_file = self.path_save + f"Data/{id_company}/PDF/{file_name}"
                    pdf = pdfplumber.open(path_of_file)
                    page = pdf.pages[0]
                    text = page.extract_text()
                    # print(text)
                    text_replace = text.replace(" ", "")
                    idx_key = text_replace.find('配当の状況')
                    
                    if idx_key != -1:
                        return self.get_dividend_table_dat(year, text, text_replace, idx_key)
                    else:
                        ocr_file = path_of_file.replace(".pdf", "_ocr.pdf")
                        if os.path.exists(ocr_file) == False:
                            ocr_pdf(path_of_file)
                        pdf = pdfplumber.open(ocr_file)
                        page = pdf.pages[0]
                        text = page.extract_text()
                        text_replace = text.replace(" ", "")
                        idx_key = text_replace.find('配当の状況')
                        if idx_key != -1:
                            return self.get_dividend_table_dat(year, text, text_replace, idx_key)
                        
                except:
                    return pd.DataFrame(
                                {year: year, "Q1": "B", "Q2": "B", "Q3": "B", "Q4": "B"}, index=[0]
                            )

        return pd.DataFrame(
            {year: year, "Q1": "N/A", "Q2": "N/A", "Q3": "N/A", "Q4": "N/A"}, index=[0]
        )
    def get_time_dividend(self, time_text):
        if len(time_text) < 3:
            return 0
        matches = re.findall(r"\d+", time_text)
        if time_text[0] == "平" :
            if time_text[2] == '元' and len(matches) == 2:
                return [
                    f"{int(matches[1])}_{int(matches[0])}_{2019}"
                ]
            if len(matches) == 3:
                return [
                        f"{int(matches[2])}_{int(matches[1])}_{int(matches[0]) + 1988 }"
                    ]
            if len(matches) == 4:
                return [
                        f"{int(matches[3])}_{int(matches[2])}_{int(matches[0])*10 + int(matches[1]) + 1988 }"
                    ]
        if time_text[0] == "令" :
            if time_text[2] == '元' and len(matches) == 2:
                return [
                    f"{int(matches[1])}_{int(matches[0])}_{2019}"
                ]
            if len(matches) == 3:
                return [
                        f"{int(matches[2])}_{int(matches[1])}_{int(matches[0]) + 2018 }"
                    ]
            if len(matches) == 4:
                return [
                        f"{int(matches[3])}_{int(matches[2])}_{int(matches[0])*10 + int(matches[1]) + 1988 }"
                    ]
        if len(matches) == 3:
            if int(matches[0]) > 2000:
                return [f"{int(matches[2])}_{int(matches[1])}_{int(matches[0])}"]
        return 0

    def get_data_from_pdf(self, 
                          id_company:int = 1301, 
                          year: str = '2022', 
                          quy: str = 'Q4'):
        # print(f'{year}_{quy}')
        try:
            for file in os.listdir(self.path_save + f"Data/{id_company}/PDF"):
                if file.startswith(f"{year}_{quy}") and "(訂正)" not in file:
                    file_name = file
                    path_of_file = self.path_save + f"Data/{id_company}/PDF/{file_name}"
                    pdf = pdfplumber.open(path_of_file)
                    page = pdf.pages[0]
                    text = page.extract_text()
                    # text = convert_pdf_to_text(path_of_file)
                    text = text.replace(" ", "")
                    for tex in ["配当支払開始予定日", '配支当払開始予定日', '配文当払開始予定日']:
                        idx = text.find(tex)
                        if idx != -1:
                            break
                    if idx != -1:
                        text__ = text[idx + 9 : ]
                        time_text = text__[0 : text__.find('\n')]
                        temp = self.get_time_dividend(time_text)
                        if temp:
                            return temp
                        else:
                            return time_text
                    else:
                        ocr_file = path_of_file.replace(".pdf", "_ocr.pdf")
                        if os.path.exists(ocr_file) == False:
                            ocr_pdf(path_of_file)
                        pdf = pdfplumber.open(ocr_file)
                        page = pdf.pages[0]
                        text = page.extract_text()
                        # text = convert_pdf_to_text(path_of_file)
                        text = text.replace(" ", "")
                        for tex in ["配当支払開始予定日", '配支当払開始予定日', '配文当払開始予定日']:
                            idx = text.find(tex)
                            if idx != -1:
                                break
                        if idx!= -1:
                            text__ = text[idx + 9 : ]
                            time_text = text__[0 : text__.find('\n')]
                            temp = self.get_time_dividend(time_text)
                            if temp:
                                return temp
                            else:
                                return time_text # xuất ra thời gian chưa qua xử lý (nếu có)
                            
                    #     if os.path.exists(ocr_file) == False:
                    #         ocr_pdf(path_of_file)
                    #     text_ = convert_pdf_to_text(ocr_file)
                    #     print(2, text_) #----------------------------
                    #     text_ = text_.replace(" ", "")

                    #     idx = text_.find("配当支払開始予定日")
                    #     if idx != -1:
                    #         _ = text_.find("四半期報告書提出予定日")
                    #         if text_[_ + 11 : _ + 14] == "\n配当":
                    #             return ["B"]

                    #         time_text = text_[idx + 9 : idx + 20]
                    #         temp = self.get_time_dividend(time_text)
                    #         if temp:
                    #             # print(1)
                    #             return temp

                    #     # case 1:
                    #     idx = text_.find("配当文払開始予定日")
                    #     if idx != -1:
                    #         time_text = text_[idx + 9 : idx + 20]
                    #         temp = self.get_time_dividend(time_text)
                    #         if temp:
                    #             # print(2)
                    #             return temp

                    #     # Case 2:
                    #     if text_.find("配当") != -1:
                    #         idx = text_.find("TEL")
                    #         time_text = text_[idx + 16 : idx + 27]
                    #         temp = self.get_time_dividend(time_text)
                    #         if temp:
                    #             # print(3)
                    #             return temp

                    return ["_"] # không dọc được file hoặc không tìm thấy key word
            return ["N/A"]
        except:
            return ["B"]

    def get_dividend(self, 
                     id_company: int = 1301,
                     return_df=False, 
                     save_file=True):
        print(id_company, '--------')
        df = pd.read_csv(self.path_save + f"Data/{id_company}/docs/link.csv")
        df_dividend = pd.DataFrame(columns=["Year", "Q1", "Q2", "Q3", "Q4"])
        for quy in ["Q4"]:
            for id in df.index:
                year = df[f"Year"][id]
                df_dividend_year = self.get_dividend_table_from_pdf_dat(
                    id_company, year, quy
                )
                # print(df_dividend_year)
                df_dividend.loc[(len(df_dividend))] = list(df_dividend_year.iloc[0])
                # df_dividend.loc[(len(df_dividend))] = [year, '1', '1', '1', '1']

        for quy in ["Q1", "Q2", "Q3", "Q4"]:
            list_date = []
            for id in df_dividend.index:
                year = df_dividend["Year"][id]
                if len(re.findall(r"\d+", df_dividend[quy][id])) > 0:
                    date = self.get_data_from_pdf(id_company, year, quy)
                    # print(year, quy, date)
                else:
                    date = np.nan
                list_date.append(date)
            df_dividend[f"time_split_{quy}"] = list_date

        if save_file:
            df_dividend.to_csv(
                self.path_save + f"Data/{id_company}/docs/dividend.csv", index=False
            )
        if return_df:
            return df_dividend


In [22]:
dividendClass = GetDividend(path_save='')
id_company = 1433

In [23]:
dividendClass.get_dividend_table_from_pdf_dat(id_company, '2015', 'Q4')

2. 配当 の 状況
年 間 配当 金 配当 金 総額 wie | MAE
第 1 四半 期末 | 第 2 四半 期末 | 第 3 四半 期末 期末 合計 (合計 ) 率
円 銭 円 銭 Fe 円 銭 Ae 百 万 円 % %
27471 A — 0.00 一 0.00 0.00 — 一 —
28年 1 月 期 — 0.00 ーー 90.00 90.00 122 35.0 6.9
29年 1 月 期 (予想 ) — 10.00 — 30.00 40.00 34.9
( 注 )1 平成 28 年 1 月 期 期末 配当 金 の 内 訳 普通 配当 70 円 00 銭 記念 配当 20円 00 銭 _-
2 当社 は 、 平 成 27 年 5 月 12 日 付 で 普通 株 式 1 株 に つき 4 株 の 株 式 分 割 を 行っ て お り ま す 。 平成28 年 1 月 期 の 期末 1 株 当たり の 配当 金 に つい て は 、 当該
株 式 分 割 後 の 金 額 を 記載 し て お り ま す 。
3 当社 は 、 平成 2 8 年 2 月 1 日 付 で 普通株 式 1 株 に つき 2 株 の 株 式 分 割 を 行っ て お り ま す 。 平成28 年 1 月 期 は 株 式 分 割 前 、 平成 29 年 1 月 期 (予想 ) は 株 式
分 割 後 の 金 額 を 記載 し て お り ま す 。
3. 平成 29 年 1 月 期 の 業績 予想 (平成2 8 年 2 月 1 日 > 平成2 9 年 1 月 31 日 )
(% 表 示 は 、 通 期 は 対 前 期 、 四 半期 は 対 前 年 同 四 半期 増減 率 )
売上 高 営業 利益 経常 利益 当期 純 利益 at rales
百 万 円 % 百 万 円 % BAR % 百 万 円 % 円 銭
第 2 四半 期 ( 累 計 ) — 一 — — 一 ーー ーー _ ーー
通期 4.700 22.2 485 8.4 488 5.1 313 7.0 114.67
( 注 )1 第 2 四半 期 (累計 ) の 業績 予想 は 行っ て お り ま せん 。 _
2 当社 は 、 平成 2 8 年 2 月 1 日 付 で 普通株 式 1 株 に つき 2 株 の 株 式 分 割 を

,2015,Q1,Q2,Q3,Q4
0,2015,(予想,),—,10.00


In [ ]:
dividendClass.get_dividend(id_company= 9982,
                     return_df=True, 
                     save_file= False)

In [ ]:
# df_cl = pd.read_csv('Docs/List_company_23052023 - Listing.csv')
# df_cl = df_cl.drop('dividend', axis=1)
# df_cl.to_csv('Docs/List_company_23052023 - Listing.csv', index=False)

#1377 2007.pdf bảng chỉ có 2 cột quý 2 và cả năm
#1433 2015 lỗi đọc sai dòng (dọc xuống dòng cuối)- do lỗi đọc file --chịu

In [7]:
pd.read_csv(f"docs/DividendShares.csv")

,Symbol,DividendShares
0,1301,False
1,1332,False
2,1333,False
3,1375,False
4,1376,False
...,...,...
1330,9994,Done
1331,9995,Done
1332,9996,Done
1333,9997,Done
